# Operaciones morfológicas<a class="tocSkip">
## TRATAMIENTO DE SEÑALES <a class="tocSkip">
### Ingenieria Electrónica <a class="tocSkip">
### Universidad Popular del Cesar <a class="tocSkip">
### Prof.: Jose Ramón Iglesias Gamarra - [https://github.com/joseramoniglesias/](https://github.com/joseramoniglesias/) <a class="tocSkip">
  **joseiglesias@unicesar.edu.co**

Las operaciones morfológicas son un conjunto amplio de operaciones que sirven para procesar imágenes basadas en formas geométricas. En una operación morfológica, cada píxel de la imagen se ajusta en función del valor de otros píxeles de su entorno. Si se seleccionan la forma y el tamaño del entorno adecuadamente, se puede definir una operación morfológica sensible a ciertas formas de la imagen de entrada [[1]](https://es.mathworks.com/help/images/morphological-filtering.html).

Las operaciones morfológicas más básicas son la dilatación y la erosión. La dilatación añade píxeles a los límites de los objetos de una imagen, mientras que la erosión elimina píxeles de los límites de los objetos. El número de píxeles que se añade o se elimina de los objetos de una imagen depende del tamaño y la forma del _elemento estructurante_ que se utiliza para procesar la imagen. En las operaciones de dilatación y erosión morfológicas, el estado de cualquier píxel de la imagen de salida se determina aplicando una regla al píxel correspondiente y a sus vecinos de la imagen de entrada. La regla utilizada para procesar los píxeles define la operación como una dilatación o una erosión [[2]](https://es.mathworks.com/help/images/morphological-dilation-and-erosion.html).

La siguiente figura muestra el efecto de la operación de erosión con diferentes elementos estructurantes ($B$):

<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/erosion.png?raw=true" alt="Erosión" style="height: 400px; width:500px;"/>

La siguiente figura muestra el efecto de la operación de dilatación con diferentes elementos estructurantes ($B$):

<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/dilatacion.png?raw=true" alt="Dilatación" style="height: 500px; width:500px;"/>

## Configuración del ambiente

# Descarga de las imágenes con las que vamos a trabajar

In [ ]:
# Importamos las librerías necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import remove_small_objects, convex_hull_image, skeletonize, binary_erosion

%matplotlib inline

def plot_images(img, title=None, font_size=None, axis="off", color=cv2.COLOR_BGR2RGB):
    n_imgs = len(img)
    if  n_imgs > 1:
        _, axs = plt.subplots(1, n_imgs, **{'figsize':(3*n_imgs, 3)})
        axs = axs.ravel()
        for i in range(n_imgs):
            if title and (len(title) == n_imgs):
                axs[i].set_title(title[i], fontsize=font_size)
            axs[i].axis(axis)
            axs[i].imshow(cv2.cvtColor(img[i], color))
        plt.tight_layout()    
    else:
        plt.title(title, fontsize=font_size)
        plt.axis(axis)
        plt.imshow(cv2.cvtColor(img[0], color))

In [ ]:
# Leemos las imágenes y nos aseguramos de que sean binarias
_, binary_shape = cv2.threshold(cv2.imread("shape2.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
_, binary_paw = cv2.threshold(cv2.imread("paw.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
_, binary_raptor = cv2.threshold(cv2.imread("raptor.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
_, binary_triangle = cv2.threshold(cv2.imread("triangle.jpg", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
binary_triangle = cv2.bitwise_not(binary_triangle)
_, binary_circuit = cv2.threshold(cv2.imread("circuit.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
pcb = cv2.imread("pcb.png", cv2.IMREAD_GRAYSCALE)
plot_images([binary_shape, binary_paw, binary_raptor, binary_triangle, binary_circuit, pcb])

In [ ]:
# Algunos elementos estructurantes
rect = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
elipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
cruz = cv2.getStructuringElement(cv2.MORPH_CROSS, (29, 29))
plot_images([255*rect, 255*elipse, 255*cruz], title=['Rectángulo', 'Elipse', 'Cruz'], axis='on')

## Operaciones Básicas
### Erosión
La erosión asigna al píxel en $(i,j)$ el valor de intensidad mínimo entre todos los píxeles de la región centrada en $(i,j)$. El elemento estructurante define la forma y tamaño de la región. 

In [ ]:
def zero_padding(img, pad_size):
    '''
    Esta función recibe una imagen y el tamaño del padding a cada lado (int).

    Retorna una nueva imagen con el contenido de la imagen original y padding cero en los bordes.
    '''
    # Dimensiones de la imagen original
    old_image_height, old_image_width = img.shape
    # Dimensiones de la imagen con padding
    new_image_width = old_image_width + 2*pad_size
    new_image_height = old_image_width + 2*pad_size
    pad_value = 0 # Valor de padding
    # Creación de la nueva imagen
    result = np.full((new_image_height, new_image_width), pad_value, dtype=np.uint8)
    x_ = (new_image_width - old_image_width) // 2 # Cálculo del desplazamiento de la imagen en el eje horizontal
    y_ = (new_image_height - old_image_height) // 2 # Cálculo del desplazamiento de la imagen en el eje vertical
    result[y_:y_+old_image_height, 
        x_:x_+old_image_width] = img # Centrar la imagen original sobre la imagen con padding
    return result

def erosion(img, struct_elem, padding_func):
    '''
    Esta función está basada en la de convolución trabajada en clase.
    Recibe una imagen, un elemento estructurante y la función de padding.

    Retorna una nueva imagen resultado de erosionar la imagen.
    '''
    # Tamaño del padding a cada lado de la imagen
    pad_size = (struct_elem.shape[0]-1)//2
    # Llamada a la función de padding
    padded_img = padding_func(img, pad_size)
    # Dimensiones de la imagen
    height, width = img.shape
    # Creación de la matriz para la nueva imagen
    new_img = np.zeros((height, width), dtype=np.uint8)
    for i in range(height): # Iterar sobre las filas
        for j in range(width): # Iterar sobre las columnas
            current_i = pad_size + i # Empezar a recorrer la imagen en el primer píxel diferente de cero
            current_j = pad_size + j
            img_region = padded_img[current_i-pad_size:current_i+pad_size+1,
                                    current_j-pad_size:current_j+pad_size+1] # Ir seleccionando las regiones de la
                                                                             # imagen para aplicar la suma de productos
            # Llamada a la función de suma de productos
            new_img[i,j] = np.min(img_region*struct_elem)
    return new_img

imageOut = erosion(binary_triangle, rect, zero_padding)
plot_images([binary_triangle, imageOut], title = ['Original', 'Erosionada'])

In [ ]:
# Utilizando la función de OpenCV
imageOut_rect = cv2.erode(binary_triangle, rect, iterations=1)
imageOut_elip = cv2.erode(binary_triangle, elipse, iterations=1)
imageOut_cruz = cv2.erode(binary_triangle, cruz, iterations=1)
plot_images([binary_triangle, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Erosionada con rectangulo', 'Erosionada con elipse', 'Erosionada con cruz'])

In [ ]:
imageOut_rect = cv2.erode(binary_triangle, rect, iterations=2)
imageOut_elip = cv2.erode(binary_triangle, elipse, iterations=2)
imageOut_cruz = cv2.erode(binary_triangle, cruz, iterations=2)
plot_images([binary_triangle, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', '2x Erosionada con rectangulo', '2x Erosionada con elipse', '2x Erosionada con cruz'])

In [ ]:
imageOut_rect = cv2.erode(binary_circuit, rect, iterations=1)
imageOut_elip = cv2.erode(binary_circuit, elipse, iterations=1)
imageOut_cruz = cv2.erode(binary_circuit, cruz, iterations=1)
plot_images([binary_circuit, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Erosionada con rectangulo', 'Erosionada con elipse', 'Erosionada con cruz'])

In [ ]:
rect_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
imageOut = cv2.erode(imageOut_rect, rect_1, iterations=1)
plot_images([binary_circuit, imageOut_rect, imageOut], title = ['Original', 'Erosionada con rectangulo 15x15', 'Segunda erosión con rectangulo 3x3'])

### Dilatación
La erosión asigna al píxel en $(i,j)$ el valor de intensidad máximo entre todos los píxeles de la región centrada en $(i,j)$. La dilatación agranda regiones "brillantes" y reduce el tamaño de regiones oscuras.

In [ ]:
def dilation(img, struct_elem, padding_func):
    '''
    Esta función está basada en la de convolución trabajada en clase.
    Recibe una imagen, un elemento estructurante y la función de padding.

    Retorna una nueva imagen resultado de dilatar la imagen.
    '''
    # Tamaño del padding a cada lado de la imagen
    pad_size = (struct_elem.shape[0]-1)//2
    # Llamada a la función de padding
    padded_img = padding_func(img, pad_size)
    # Dimensiones de la imagen
    height, width = img.shape
    # Creación de la matriz para la nueva imagen
    new_img = np.zeros((height, width), dtype=np.uint8)
    for i in range(height): # Iterar sobre las filas
        for j in range(width): # Iterar sobre las columnas
            current_i = pad_size + i # Empezar a recorrer la imagen en el primer píxel diferente de cero
            current_j = pad_size + j
            img_region = padded_img[current_i-pad_size:current_i+pad_size+1,
                                    current_j-pad_size:current_j+pad_size+1] # Ir seleccionando las regiones de la
                                                                             # imagen para aplicar la suma de productos
            # Llamada a la función de suma de productos
            new_img[i,j] = np.max(img_region*struct_elem)
    return new_img

imageOut = dilation(binary_triangle, rect, zero_padding)
plot_images([binary_triangle, imageOut], title = ['Original', 'Dilatada'])

In [ ]:
# Utilizando la función de OpenCV
imageOut_rect = cv2.dilate(binary_triangle, rect, iterations=1)
imageOut_elip = cv2.dilate(binary_triangle, elipse, iterations=1)
imageOut_cruz = cv2.dilate(binary_triangle, cruz, iterations=1)
plot_images([binary_triangle, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Dilatada con rectangulo', 'Dilatada con elipse', 'Dilatada con cruz'])

In [ ]:
imageOut_rect = cv2.erode(binary_paw, rect, iterations=1)
imageOut_elip = cv2.erode(binary_paw, elipse, iterations=1)
imageOut_cruz = cv2.erode(binary_paw, cruz, iterations=1)
plot_images([binary_paw, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Dilatada con rectangulo', 'Dilatada con elipse', 'Dilatada con cruz'])

Es importante notar, como los puntos negros que teníamos en la imagen original toman la forma del elemento estructurante usado en la dilatación.

### Apertura
La apertura morfólogica consiste en una erosión seguida de una dilatación. La apertura puede remover pequeños puntos blancos (el ruido que llamamos sal) y conectar dos regiones oscuras.

In [ ]:
imageOut_1 = cv2.erode(binary_shape, rect, iterations=1)
imageOut = cv2.dilate(imageOut_1, rect, iterations=1)
plot_images([binary_shape, imageOut_1, imageOut], title = ['Original', 'Erosionada', 'Dilatada (Apertura)'])

In [ ]:
imageOut_rect = cv2.morphologyEx(binary_shape, cv2.MORPH_OPEN, rect)
imageOut_elip = cv2.morphologyEx(binary_shape, cv2.MORPH_OPEN, elipse)
imageOut_cruz = cv2.morphologyEx(binary_shape, cv2.MORPH_OPEN, cruz)
plot_images([binary_shape, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Apertura con rectangulo', 'Apertura con elipse', 'Apertura con cruz'])

### Cierre
El cierre se define como una dilatación seguida de una erosión. El cierre puede remover pequeños puntos oscuros (el ruido que llamamos pimienta) y conectar dos regiones claras.

In [ ]:
imageOut_1 = cv2.dilate(binary_paw, rect, iterations=1)
imageOut = cv2.erode(imageOut_1, rect, iterations=1)
plot_images([binary_paw, imageOut_1, imageOut], title = ['Original', 'Dilatada', 'Erosionada (Cierre)'])

Cambiando el elemento estructurante, podemos obtener mejores resultados dada la forma de la figura.

In [ ]:
elipse_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
imageOut_1 = cv2.dilate(binary_paw, elipse_1, iterations=1)
imageOut = cv2.erode(imageOut_1, elipse_1, iterations=1)
plot_images([binary_paw, imageOut_1, imageOut], title = ['Original', 'Dilatada', 'Erosionada (Cierre)'])

In [ ]:
imageOut_rect = cv2.morphologyEx(binary_paw, cv2.MORPH_CLOSE, rect)
imageOut_elip = cv2.morphologyEx(binary_paw, cv2.MORPH_CLOSE, elipse) # Este elemento estructurante es más grande que el del ejemplo anterior
imageOut_cruz = cv2.morphologyEx(binary_paw, cv2.MORPH_CLOSE, cruz)
plot_images([binary_paw, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Cierre con rectangulo', 'Cierre con elipse', 'Cierre con cruz'])

## Algunas aplicaciones
### Detección de características
<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/deteccionCaracteristicas.png?raw=true" alt="Dilatación" style="height: 400px; width:650px;"/>

In [ ]:
def match_struct(img, struct_elem, padding_func):
    '''
    Esta función está basada en la de convolución trabajada en clase.
    Recibe una imagen, un elemento estructurante y la función de padding.

    Retorna una nueva imagen resultado de dilatar la imagen.
    '''
    # Tamaño del padding a cada lado de la imagen
    pad_size = (struct_elem.shape[0]-1)//2
    # Llamada a la función de padding
    padded_img = padding_func(img, pad_size)
    # Dimensiones de la imagen
    height, width = img.shape
    # Creación de la matriz para la nueva imagen
    new_img = np.zeros((height, width), dtype=np.uint8)
    for i in range(height): # Iterar sobre las filas
        for j in range(width): # Iterar sobre las columnas
            current_i = pad_size + i # Empezar a recorrer la imagen en el primer píxel diferente de cero
            current_j = pad_size + j
            img_region = padded_img[current_i-pad_size:current_i+pad_size+1,
                                    current_j-pad_size:current_j+pad_size+1] # Ir seleccionando las regiones de la
                                                                             # imagen para aplicar la suma de productos
            # Llamada a la función de suma de productos
            new_img[i,j] = 255*np.all(img_region==struct_elem)
    return new_img

In [ ]:
lower_edge = np.array([[0, 255, 255],
                       [0, 255, 255],
                       [0,   0,   0]], dtype=np.uint8)
imageOut = match_struct(binary_triangle, lower_edge, zero_padding)
plot_images([binary_triangle[285:295, 40:50], imageOut[285:295, 40:50]], title = ['Original', 'Coincidencia'])

In [ ]:
left_border = np.array([[0, 255, 255],
                        [0, 255, 255],
                        [0, 255, 255]], dtype=np.uint8)
imageOut = match_struct(binary_triangle, left_border, zero_padding)
plot_images([binary_triangle[:, 30:80], imageOut[:, 30:80]], title = ['Original', 'Coincidencia'])

### Extracción de bordes
<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/extraccionBordes.png?raw=true" alt="Dilatación" style="height: 300px; width:450px;"/>

In [ ]:
imageOut_1 = cv2.erode(binary_triangle, rect, iterations=1)
imageOut = binary_triangle - imageOut_1
plot_images([binary_triangle, imageOut_1, imageOut], title = ['Original', 'Erosionada', 'Bordes'])

In [ ]:
rect_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
imageOut_1 = cv2.erode(binary_raptor, rect, iterations=1)
imageOut = binary_raptor - imageOut_1
plot_images([binary_raptor, imageOut_1, imageOut], title = ['Original', 'Erosionada', 'Bordes'])

### Esqueletos

In [ ]:
copy_raptor = np.copy(binary_raptor)
copy_raptor[copy_raptor == 255] = 1 # change 255 (white) value to 1 (True)

imageOut = skeletonize(copy_raptor).astype(np.uint8)*255 # Skeletonization
plot_images([binary_raptor, imageOut], title = ['Original', 'Esqueleto'])

### Eliminación de objetos
Tenemos dos operaciones conocidas como _White Tophat_ y _Black Tophat_, que nos permiten mantener los objetos blancos y negros, respectivamente, que son más pequeños que el elemento estructurante.

In [ ]:
elipse_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
imageOut_1 = cv2.morphologyEx(binary_paw, cv2.MORPH_CLOSE, elipse_1)
imageOut = imageOut_1 - binary_paw
plot_images([binary_paw, imageOut_1, imageOut], title = ['Original', 'Cierre', 'Black Tophat (Cierre - Original)'])

In [ ]:
elipse_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
imageOut_1 = cv2.morphologyEx(binary_shape, cv2.MORPH_OPEN, elipse_1)
imageOut = binary_shape - imageOut_1
plot_images([binary_shape, imageOut_1, imageOut], title = ['Original', 'Apertura', 'White Tophat (Original - Apertura)'])

## Convex Hull
Para determinar la superficie de menor tamaño que abarca todo el objeto.

In [ ]:
imageOut = convex_hull_image(copy_raptor).astype(np.uint8)*255
plot_images([binary_raptor, imageOut], title = ['Original', 'Convex Hull'])

## Operaciones morfólogicas en imágenes en escala de grises

In [ ]:
imageOut_rect = cv2.erode(pcb, rect, iterations=1)
imageOut_elip = cv2.erode(pcb, elipse, iterations=1)
imageOut_cruz = cv2.erode(pcb, cruz, iterations=1)
plot_images([pcb, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Dilatada con rectangulo', 'Dilatada con elipse', 'Dilatada con cruz'])

In [ ]:
imageOut_rect = cv2.dilate(pcb, rect, iterations=1)
imageOut_elip = cv2.dilate(pcb, elipse, iterations=1)
imageOut_cruz = cv2.dilate(pcb, cruz, iterations=1)
plot_images([pcb, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Dilatada con rectangulo', 'Dilatada con elipse', 'Dilatada con cruz'])

In [ ]:
imageOut_rect = cv2.morphologyEx(pcb, cv2.MORPH_OPEN, rect)
imageOut_elip = cv2.morphologyEx(pcb, cv2.MORPH_OPEN, elipse)
imageOut_cruz = cv2.morphologyEx(pcb, cv2.MORPH_OPEN, cruz)
plot_images([pcb, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Apertura con rectangulo', 'Apertura con elipse', 'Apertura con cruz'])

In [ ]:
imageOut_rect = cv2.morphologyEx(pcb, cv2.MORPH_CLOSE, rect)
imageOut_elip = cv2.morphologyEx(pcb, cv2.MORPH_CLOSE, elipse)
imageOut_cruz = cv2.morphologyEx(pcb, cv2.MORPH_CLOSE, cruz)
plot_images([pcb, imageOut_rect, imageOut_elip, imageOut_cruz], title = ['Original', 'Apertura con rectangulo', 'Apertura con elipse', 'Apertura con cruz'])

### Reducción de ruido

In [ ]:
def ruido_sal_pimienta(img, noise_prob):
    '''
    Esta función adicionan ruido sal y pimimienta a una imagen con base en una probabilidad de ruido (noise_prob),
    a mayor probabilidad mayor ruido.

    Una forma de implementarlo es generando una matriz aleatoria del tamaño de la imagen y utilizar la probabilidad como un umbral.

    Retorna la imagen con ruido.
    '''
    new_img = np.copy(img)
    h, w = img.shape
    for i in range(h):
        for j in range(w):
            aleatorio = np.random.randint(0,100)
            if aleatorio < noise_prob:
                new_img[i,j] = 255
            elif aleatorio > (100-noise_prob):
                new_img[i,j] = 0
    return new_img
noisy_image = ruido_sal_pimienta(pcb, 2)
plot_images([pcb, noisy_image], title = ['Original', 'Con Ruido'])

In [ ]:
rect_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
imageOut_1 = cv2.morphologyEx(noisy_image, cv2.MORPH_CLOSE, rect_1)
imageOut = cv2.morphologyEx(imageOut_1, cv2.MORPH_OPEN, rect_1)
plot_images([noisy_image, imageOut_1, imageOut], title = ['Original', 'Cierre', 'Apertura'])

## Pruebas

In [ ]:
_, binary_dots = cv2.threshold(cv2.imread("dots.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
_, binary_finger = cv2.threshold(cv2.imread("huella.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
phantom = cv2.imread("phantom.png", cv2.IMREAD_GRAYSCALE)
plot_images([binary_dots, binary_finger, phantom])

**Copyright**

The notebooks are provided as [Open Educational Resource](https://de.wikipedia.org/wiki/Open_Educational_Resources). Feel free to use the notebooks for your own educational purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).